In [5]:
import hashlib 
def compute_hash(path, algorithm): 
    h = hashlib.new(algorithm) 
    with open(path, "rb") as f: 
        h.update(f.read()) 
    return h.hexdigest() 
sample = r"C:\Users\blabb\Portfolio\Procmon.exe" 
print("MD5:   ", compute_hash(sample, "md5")) 
print("SHA1:  ", compute_hash(sample, "sha1")) 
print("SHA256:", compute_hash(sample, "sha256"))

MD5:    3b493c33bc35f2b0ecf3c354d58131d6
SHA1:   f92ab0bebde636ece6f7bb191d3b2b12c682c085
SHA256: 883b2f76712d9c3edd2bfd51443d37b62ab66c5b637d161753465ff912a86043


In [6]:
import re 
def extract_strings(path): 
    with open(path, "rb") as f: 
        data = f.read() 
    pattern = rb"[ -~]{4,}" 
    return re.findall(pattern, data) 
strings = extract_strings(sample) 
for s in strings[:20]: 
    print(s.decode(errors="ignore")) 

!This program cannot be run in DOS mode.
V*0T
0RichU
.text
`.rdata
@.data
.rsrc
@.reloc
hpqQ
h`EN
h|nN
h\nN
hlnN
=UUU
h_rM
hDLN
h`GO
hDLN
h|GO
hDLN


In [ ]:
import pefile 
pe = pefile.PE(sample) 
print("Entry Point:", hex(pe.OPTIONAL_HEADER.AddressOfEntryPoint)) 
print("Image Base:", hex(pe.OPTIONAL_HEADER.ImageBase)) 
print("\nImported DLLs and functions:") 
for entry in pe.DIRECTORY_ENTRY_IMPORT: 
    print(" ", entry.dll.decode()) 
    for imp in entry.imports[:5]: 
        print("     -", imp.name.decode() if imp.name else "None") 

This code successfully parses the structure of a PE file to extract key information about its execution and dependencies. Specifically, it retrieves the program's Entry Point and Image Base memory addresses, and then lists the DLLs the executable imports, along with the first few functions required from each.

In [ ]:
import yara 
rule_source = """ 
rule ContainsHTTP { 
strings: 
$s = "http" 
condition: 
$s 
} 
""" 
rules = yara.compile(source=rule_source) 
matches = rules.match(sample) 
print(matches) 

In [ ]:
import hashlib, pefile, re, yara 
# sample = "samples/procmon.exe" 
def compute_hashes(path): 
    algos = ["md5", "sha1", "sha256"] 
    output = {} 
    for a in algos: 
        h = hashlib.new(a) 
        with open(path, "rb") as f: 
            h.update(f.read()) 
        output[a] = h.hexdigest() 
    return output 
def extract_strings(path): 
    with open(path, "rb") as f: 
        data = f.read() 
    return re.findall(rb"[ -~]{4,}", data) 
print("Hashes:", compute_hashes(sample)) 
print("\nStrings:") 
print(extract_strings(sample)[:10]) 
print("\nImports:") 
pe = pefile.PE(sample) 
for entry in pe.DIRECTORY_ENTRY_IMPORT: 
    print(entry.dll.decode()) 
print("\nIOCs:") 
decoded = open(sample, "rb").read().decode(errors="ignore") 
print("URLs:", re.findall(r"https?://[^\s\"']+", decoded)) 
print("IPs:", re.findall(r"\b\d{1,3}(?:\.\d{1,3}){3}\b", decoded)) 
print("\nYARA:") 
rule = yara.compile(source=""" 
rule Simple { 
strings: $s = "http" 
condition: $s 
} 
""") 
print(rule.match(sample))

The script performs a series of static analysis steps on the target file, including calculating its cryptographic hashes for identification and threat intelligence lookups, extracting readable strings to uncover potential data or commands, and using the pefile library to list the executable's required DLL imports. Finally, it extracts potential Indicators of Compromise (IOCs) like URLs and IP addresses using regular expressions and utilizes YARA rules for pattern matching and signature detection.